In [1]:
dataset_path = "/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_3/YOLO_anno"
yaml_path = "/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_3/YOLO_anno/IndoorVG3.yaml"

model_path = "/home/maelic/Documents/PhD/MyModel/Scene-Graph-Benchmark-Cuda11.7/datasets/IndoorVG/yolov8l_indoorvg.pt"

import os
from ultralytics import YOLO
import torch

# load model
model = YOLO(model_path)

from ultralytics.cfg import get_cfg

cfg = get_cfg()

In [11]:
# load train data
# load yaml
from ultralytics.utils import yaml_load
from ultralytics.data import build_dataloader, build_yolo_dataset

data = yaml_load(yaml_path)

# loda dataset
from ultralytics.data import YOLODataset

# load dataset
splits = [data['train'], data['val'], data['test']]

split = splits[1]
img_dir = os.path.join(split, 'images')
dataset = build_yolo_dataset(cfg, img_dir, 1, data)

print(dataset)


train: Scanning /home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_3/YOLO_anno/val/labels.cache... 9847 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9847/9847 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_3/YOLO_anno/val/images/1592276.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_3/YOLO_anno/val/images/2560.jpg: corrupt JPEG restored and saved
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


# Utils

In [ ]:
new_dataset_path = "/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_3/YOLO_anno_v2"
# create dir
os.makedirs(new_dataset_path, exist_ok=True)

In [51]:
from utils import bbox_iou

iou_thres = 0.5
conf_thres = 0.5

# img_file = "/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_3/YOLO_anno/val/images/12.jpg"

# run model on dataset
for img in dataset:
    # get gt boxes
    gt_boxes = img['bboxes'].cuda()

    # forward pass
    results = model(img['im_file'])

    print(len(results))

    print(results[0].boxes)
    print(gt_boxes.shape)

    # iou on gt boxes and predictions
    for i, box in enumerate(results[0].boxes.xywh):
        iou = bbox_iou(box, gt_boxes, xywh=True)
        print(iou)
        if iou < iou_thres:
            # the box is a candidate for a new object
            if box.conf[i] > conf_thres:
                # save the box
                pass
    
    break

image 1/1 /home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_3/YOLO_anno/val/images/1001.jpg: 480x640 (no detections), 19.8ms
Speed: 2.4ms preprocess, 19.8ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
1
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (600, 800)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))
torch.Size([2, 4])
